# Kliment Mamykin, UNI 2770
## Algorythms for Data Science, Homework 1

### Problem 1(a)

**Process:** restate the algorythm, establish loop invariant and prove the correctness by showing that the loop invariant holds at the initialization before the loop, after each iteration of the loop, and after the termination.

**Insight:** a polynomial of order $n$ $$
p ( x ) = a_{0} + a_{1} x + a_{2} x^{2} + \ldots + a_{n} x^{n} = \sum_{k=0}^{n} {a_{k}x^k}
$$ can be re-written as 

\begin{equation}
p ( x ) = a_{0} + (a_{1} + (a_{2} + \ldots (a_{n-1} + (a_{n})x) \ldots )x)x
\end{equation}


**Loop invariant**: At the start of each $i$ loop, $z$ contains the value of polynomial of order $n-(i+1)$ with coefficients $A[i+1..n]$. Mathematically 
$$
z_i = \sum_{k=0}^{n-(i+1)} {a_{k+i+1}x^k}
$$

**Initialization:** We need to prove that for $i = n-1$ the initialization value of $z=A[n]$ satisfies the loop invariant. 

$z$ contains the value of the polinomial of order $n - i - 1 = n - (n - 1) - 1 = 0$ with coefficients $A[i+1..n] = A[(n-1)+1..n] = A[n]$. Therefor initialization of $z = A[n]$ satisfies the loop invariant. 

$$
z_{n-1} = \sum_{k=0}^{n-(n-1+1)} {a_{k+(n-1)+1}x^k} = a_{n}x^0 = a_n
$$

**Maintenance:** At each iteration of the loop $z$ is assigned a new value to be used on the next loop iteration $z_{i-1}$ (loop is counting down) based on the value before the loop iteration $z_i$. At a particular interation $i$ we have 

$$
z_{i-1} = a_i + z_{i} x = a_i + (\sum_{k=0}^{n-(i+1)} {a_{k+i+1}x^k}) x = \sum_{k=0}^{n-i} {a_{k+i}x^k}  = \sum_{k=0}^{n-((i-1)+1)} {a_{k+(i-1)+1}x^k}
$$

After the loop completes, $z_{i-1}$ for the next iteration of the loop satisfies teh invariant.

**Termination:** Eventually $i$ will be assigned the value $-1$, the condition of the _for_ loop fails and loop terminates. At this point $z$ contain the value of the last loop iteration at $i=0$. We need to prove that at this point $z = \sum_{k=0}^{n} {a_{k}x^k}$ based on the invariant.

$$
z = \sum_{k=0}^{n-(-1+1)} {a_{k+(-1)+1}x^k} = \sum_{k=0}^{n} {a_{k}x^k}
$$

### Problem 1(b)

Horner's rule uses $n$ multiplications and $n$ additions. Each iteration of the loop from $n-1$ to $0$ ($n$ iterations performs one addition and one multiplication.

Even for a polynomial with large $n$ and all lower terms with 0 coefficients $A = [0,0,\ldots,0,a_n]$ the naive calculation could shirtcut multiplication by 0, but the check for 0 for each coefficient would still take constant time and the naive implementation will not be more efficient then Horner's rule implementation.

### Problem 2(a)


**Correctness proof**

First use the fact that for any matrix $A$, its product with a vector $v$ can be expressed through blocks of partitioned matrix $A$ and partitioned vector $v$. This is a special case of a partitioned (block) matrices product rule (see ref 1).

$$
\begin{equation}
A v 
= 
\left[ \begin{array} { c | c } A_{11} & A_{12} \\ \hline A_{21} & A_{22} \end{array} \right] 
\left[ \begin{array} { c } v_{1} \\ \hline v_{2} \end{array} \right]
= 
\left[ \begin{array} { c } A_{11} v_{1} + A_{12} v_{2} \\ \hline A_{21} v_{1} + A_{22} v_{2} \end{array} \right]
\end{equation}
$$


Now we prove by induction the correctness of the `Hadamand(v)` algorithm.

Let $v^{h}$ and $v^{l}$ be the high and the low part of the vector $v$ partitioned into 2 halves.

**Proposition** $P(k)$ - this is the essence of the proposed algorithm, which recursively calculates lower order Hadamand product of high and low halves of vector $v$ and then computes the sum and difference of those vectors.  

$$
H_{k} v = \left[ \begin{array} { c } H_{k-1} v^{h} + H_{k-1} v^{l} \\ H_{k-1} v^{h} - H_{k-1} v^{l} \end{array} \right], \forall k > 0
$$

**Base Case:** for $k = 1$, matrix $H_1$ is size 2, vector $v$ is size 2. Matrix $H_0 = [1]$ by definition of Hadamand matrix.
$$
H_{1} v 
= 
\left[ \begin{array} { r r } 1 & 1 \\ 1 & -1 \end{array} \right] 
\left[ \begin{array} { c } v_{1} \\ v_{2} \end{array} \right]
= 
\left[ \begin{array} { c } v_{1} + v_{2} \\ v_{1} - v_{2} \end{array} \right]
= 
\left[ \begin{array} { c } H_{0} v_{1} + H_{0} v_{2} \\ H_{0} v_{1} - H_{0} v_{2} \end{array} \right]
$$

**Inductive step:** for $k > 1$ assume $P(k-1)$ is correct. Need to prove that $P(k)$ is also correct.

$$
\begin{align}
H_{k} v 
&= 
\left[ \begin{array} { r r } H_{k-1} & H_{k-1} \\ H_{k-1} & -H_{k-1} \end{array} \right] 
\left[ \begin{array} { c } v^{h} \\ v^{l} \end{array} \right], \text { expanded Hadamand matrix definition and partitioned vector v into halves} \\
&= 
\left[ \begin{array} { c } H_{k-1} v^{h} + H_{k-1} v^{l} \\ H_{k-1} v^{h} - H_{k-1} v^{l} \end{array} \right], \text { using partitioned matrix/vector product rule } \\
\end{align}
$$

Proposition holds true for $k$.


**Running time T(n)**

line 5: $\Theta(1)$

line 6: $\Theta(n)$ - partitioning a vector involves copy of all elements in vector into 2 subvectors, done in linear time

line 7: $T(n/2)$ - recursive call on a vector with half of the size

line 8: $T(n/2)$ - recursive call on a vector with half of the size

line 9: $cn/2 = \Theta(n)$ - vector addition is done in linear time

line 10: $cn/2 = \Theta(n)$ - vector subtract is done in linear time

line 11: $\Theta(n)$ - copy all elements into resulting vector of size n

Combined running time: $T(n) = 2T(n/2) + \Theta(n)$

Using Master theorem with $a = 2, b = 2, f(n) = \Theta(n)$, we conclude that the running time of our algorithm is $T(n) = O(n\log{n})$


**References**
1. Block matrix (https://en.wikipedia.org/wiki/Block_matrix)